In [3]:
from datasets import load_dataset

In [2]:
from datasets import load_dataset

archive_data = load_dataset("json", data_files="/lstr/sahara/datalab-ml/z1974769/arxiv-metadata-oai-snapshot.json")

In [3]:
cs_data = archive_data.filter(lambda data_point: data_point["categories"].startswith("cs"))

In [4]:
from collections import defaultdict
selected_papers_by_category = defaultdict(list)

for paper in cs_data["train"]:
    categories_of_paper = paper['categories'].split()
    for category in categories_of_paper:
        if category.startswith('cs.') and len(selected_papers_by_category[category]) < 100:
            selected_papers_by_category[category].append(paper)

In [12]:
selected_papers_by_category["cs.AI"]

[{'id': '0704.0047',
  'submitter': 'Igor Grabec',
  'authors': 'T. Kosel and I. Grabec',
  'title': 'Intelligent location of simultaneously active acoustic emission sources:\n  Part I',
  'comments': '5 pages, 5 eps figures, uses IEEEtran.cls',
  'journal-ref': None,
  'doi': None,
  'report-no': None,
  'categories': 'cs.NE cs.AI',
  'license': None,
  'abstract': '  The intelligent acoustic emission locator is described in Part I, while Part\nII discusses blind source separation, time delay estimation and location of two\nsimultaneously active continuous acoustic emission sources.\n  The location of acoustic emission on complicated aircraft frame structures is\na difficult problem of non-destructive testing. This article describes an\nintelligent acoustic emission source locator. The intelligent locator comprises\na sensor antenna and a general regression neural network, which solves the\nlocation problem based on learning from examples. Locator performance was\ntested on different 

In [9]:
from datasets import Dataset

papers_list = []
for category, papers in selected_papers_by_category.items():
    for paper in papers:
        paper["unique_category"] = category
        papers_list.append(paper)

cs_dataset_per_category = Dataset.from_dict({"data" : papers_list})
cs_dataset_per_category.to_json("intermediate_data.json")

Creating json from Arrow format:   0%|          | 0/4 [00:00<?, ?ba/s]

6191246

In [4]:
cs_archive_intermediate_data = load_dataset("json", data_files="intermediate_data.json")

In [5]:
cs_archive_intermediate_data["train"]

Dataset({
    features: ['data'],
    num_rows: 4000
})

In [6]:
unique_ids = set()
for paper in papers_list:
    unique_ids.add(paper["id"])
len(unique_ids)

NameError: name 'papers_list' is not defined

In [7]:
# for paper in cs_archive_intermediate_data["train"]:
#     id  = paper["data"]["id"]
#     url_template = f"https://arxiv.org/pdf/{id}.pdf"
    

import requests
import time
from concurrent.futures import ThreadPoolExecutor
from pathlib import Path

def download_paper(arxiv_id, save_dir):
    try:
        pdf_url = f"https://arxiv.org/pdf/{arxiv_id}.pdf"
        response = requests.get(pdf_url, timeout=30, stream=True)
        if response.status_code == 200:
            file_path = Path(save_dir) / f"{arxiv_id}.pdf"
            with open(file_path, "wb") as f:
                f.write(response.content)
            print(f"Downloaded {arxiv_id}")
        else:
            print(f"Failed to download {arxiv_id}")
    except requests.RequestException as e:
        print(f"Error downloading {arxiv_id}: {e}")


arxiv_ids = unique_ids
save_directory = "./cs_intermediate_papers"


with ThreadPoolExecutor(max_workers=10) as executor:
    futures = [executor.submit(download_paper, arxiv_id, save_directory) for arxiv_id in arxiv_ids]
    for future in futures:
        future.result()

print("All downloads completed.")


All downloads completed.


In [8]:
import fitz
example_doc = fitz.open("/lstr/sahara/datalab-ml/z1974769/cs_intermediate_papers/0704.0050.pdf")

askjk = []
for page_num in range(len(example_doc)):
    page = example_doc[page_num]
    
    # Extract images
    image_list = page.get_images(full=True)
    askjk.append(image_list)
    print(image_list)

[]
[(54, 0, 256, 256, 1, '', '', 'R54', 'CCITTFaxDecode', 0)]
[]
[]
[]


In [35]:
from pandas.core.common import flatten

list(flatten(askjk))

[54, 0, 256, 256, 1, '', '', 'R54', 'CCITTFaxDecode', 0]

In [10]:
import fitz
from IPython.display import display, Image
import os

pdf_path = "/lstr/sahara/datalab-ml/z1974769/cs_intermediate_papers/0704.0050.pdf"
doc = fitz.open(pdf_path)
saved_image_path = "/lstr/sahara/datalab-ml/z1974769/saved_images"

for page_num in range(len(doc)):
    page = doc.load_page(page_num)
    image_list = page.get_images(full=True)
    
    for image_index, img in enumerate(image_list):
        xref = img[0]
        base_image = doc.extract_image(xref)
        image_bytes = base_image["image"]
        image_filename = f"image_page_{page_num}_{image_index}.png"
        with open(os.path.join(saved_image_path, image_filename), "wb") as img_file:
            img_file.write(image_bytes)

In [1]:
di = {"ahjgvav":"havcab", }

('ahjgvav', 'havcab')